In [1]:
!python -V

Python 3.9.21


In [1]:
import pandas as pd

In [2]:
import pickle

In [10]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error

In [4]:
import mlflow

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='mlflow-artifacts:/1', creation_time=1749134557942, experiment_id='1', last_update_time=1749134557942, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [5]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')
print(len(df))

3403766


In [6]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

df = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')
print(len(df))


3316216


In [9]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

In [11]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

root_mean_squared_error(y_train, y_pred)

8.163436921591176

In [12]:
print(lr.intercept_)

23.90171730136149


In [20]:
import pickle

with open('homework03_models/lin_reg.bin', 'wb') as f_out:
    pickle.dump(lr, f_out)

In [21]:
with mlflow.start_run():
    lr = LinearRegression()
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_train)

    rmse = root_mean_squared_error(y_train, y_pred)

    mlflow.log_metric('rmse', rmse)
    mlflow.log_artifact(local_path='homework03_models/lin_reg.bin', artifact_path='homework03_models')

🏃 View run debonair-carp-322 at: http://localhost:5000/#/experiments/1/runs/3e5172d8fb1f41d2872cec29c5afae58
🧪 View experiment at: http://localhost:5000/#/experiments/1
